## Spark Wrangling Exercises

In [1]:
# imports
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, StringType

spark = SparkSession.builder.getOrCreate()

### Data Acquisition
- These exercises should go in a notebook or script named wrangle. Add, commit, and push your changes.
- This exercises uses the 'case.csv', 'dept.csv', and 'source.csv' files from the san antonio 311 call dataset.

1. Read the case, department, and source data into their own spark dataframes.

In [2]:
# first I need to download all three files as csv, and store locally

In [3]:
# case
# Read in CSV file 
case = (spark.read.csv("case.csv",
                     sep=",",
                     header=True,
                     inferSchema=True)
     )

In [4]:
# dept
# Read in CSV file 
dept = (spark.read.csv("dept.csv",
                     sep=",",
                     header=True,
                     inferSchema=True)
     )

In [5]:
# source
# Read in CSV file 
source = (spark.read.csv("source.csv",
                     sep=",",
                     header=True,
                     inferSchema=True)
     )

2. Let's see how writing to the local disk works in spark:
- Write the code necessary to store the source data in both csv and json format, store these as 'sources_csv' and 'sources_json'
- Inspect your folder structure. What do you notice?

In [6]:
# let's write as a csv first and document what happens...
(
    source.write.format('csv')
    .mode('overwrite')
    .option('header', 'true')
    .save('sources_csv')
)

In [7]:
# now as a json...
(
    source.write.format('json')
    .mode('overwrite')
    .option('header', 'true')
    .save('sources_json')
)

In [9]:
# It appears both files are too large to be stored locally, 
# and that only a small portion was saved

3. Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.

#### Case

In [14]:
case.show(1, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 1/1/18 0:42          
 case_closed_date     | 1/1/18 12:29         
 SLA_due_date         | 9/26/20 0:42         
 case_late            | NO                   
 num_days_late        | -998.5087616000001   
 case_closed          | YES                  
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
only showing top 1 row



In [10]:
# case
case.dtypes

[('case_id', 'int'),
 ('case_opened_date', 'string'),
 ('case_closed_date', 'string'),
 ('SLA_due_date', 'string'),
 ('case_late', 'string'),
 ('num_days_late', 'double'),
 ('case_closed', 'string'),
 ('dept_division', 'string'),
 ('service_request_type', 'string'),
 ('SLA_days', 'double'),
 ('case_status', 'string'),
 ('source_id', 'string'),
 ('request_address', 'string'),
 ('council_district', 'int')]

In [19]:
# Change 'SLA_due_date' to 'case_due_date'
# Change 'SLA_days' tp 'case_days_alotted'

case = case.withColumnRenamed('SLA_due_date', 'case_due_date')
case = case.withColumnRenamed('SLA_days', 'case_days')

In [20]:
# did it work as expected?
case.show(1, truncate = False, vertical = True)

-RECORD 0----------------------------------------------------
 case_id              | 1014127332                           
 case_opened_date     | 1/1/18 0:42                          
 case_closed_date     | 1/1/18 12:29                         
 case_due_date        | 9/26/20 0:42                         
 case_late            | NO                                   
 num_days_late        | -998.5087616000001                   
 case_closed          | YES                                  
 dept_division        | Field Operations                     
 service_request_type | Stray Animal                         
 case_days            | 999.0                                
 case_status          | Closed                               
 source_id            | svcCRMLS                             
 request_address      | 2315  EL PASO ST, San Antonio, 78207 
 council_district     | 5                                    
only showing top 1 row



In [21]:
# 'case_late' and 'case_closed' to boolean
case = case.withColumn('case_closed', expr('case_closed == "YES"'))\
.withColumn('case_late', expr('case_late == "YES"'))

In [22]:
# did it work as expected?
case.show(1, truncate = False, vertical = True)

-RECORD 0----------------------------------------------------
 case_id              | 1014127332                           
 case_opened_date     | 1/1/18 0:42                          
 case_closed_date     | 1/1/18 12:29                         
 case_due_date        | 9/26/20 0:42                         
 case_late            | false                                
 num_days_late        | -998.5087616000001                   
 case_closed          | true                                 
 dept_division        | Field Operations                     
 service_request_type | Stray Animal                         
 case_days            | 999.0                                
 case_status          | Closed                               
 source_id            | svcCRMLS                             
 request_address      | 2315  EL PASO ST, San Antonio, 78207 
 council_district     | 5                                    
only showing top 1 row



In [24]:
# council district as a string
case = case.withColumn('council_district', col('council_district').cast('string'))

In [25]:
# 'case_opened_date', 'case_closed_date', 'case_due_date' to datetime format

fmt = "M/d/yy H:mm"

case = case.withColumn('case_opened_date', to_timestamp('case_opened_date', fmt))\
.withColumn('case_closed_date', to_timestamp('case_closed_date', fmt))\
.withColumn('case_due_date', to_timestamp('case_due_date', fmt))

In [26]:
# did it work as expected?
case.show(1, truncate = False, vertical = True)

-RECORD 0----------------------------------------------------
 case_id              | 1014127332                           
 case_opened_date     | 2018-01-01 00:42:00                  
 case_closed_date     | 2018-01-01 12:29:00                  
 case_due_date        | 2020-09-26 00:42:00                  
 case_late            | false                                
 num_days_late        | -998.5087616000001                   
 case_closed          | true                                 
 dept_division        | Field Operations                     
 service_request_type | Stray Animal                         
 case_days            | 999.0                                
 case_status          | Closed                               
 source_id            | svcCRMLS                             
 request_address      | 2315  EL PASO ST, San Antonio, 78207 
 council_district     | 5                                    
only showing top 1 row



In [28]:
# trim and lowercase address info

case = case.withColumn('request_address', trim(lower(case.request_address)))

In [29]:
# use format_string function to pad zeros for council_district

case = case.withColumn('council_district', format_string('%03d', col('council_district').cast('int')))

In [30]:
# create a new column for zipcode:

case = case.withColumn('zipcode', regexp_extract('request_address', r"(\d+$)", 1))

In [42]:
case.show(1, False, True)

-RECORD 0----------------------------------------------------
 case_id              | 1014127332                           
 case_opened_date     | 2018-01-01 00:42:00                  
 case_closed_date     | 2018-01-01 12:29:00                  
 case_due_date        | 2020-09-26 00:42:00                  
 case_late            | false                                
 num_days_late        | -998.5087616000001                   
 case_closed          | true                                 
 dept_division        | Field Operations                     
 service_request_type | Stray Animal                         
 case_days            | 999.0                                
 case_status          | Closed                               
 source_id            | svcCRMLS                             
 request_address      | 2315  el paso st, san antonio, 78207 
 council_district     | 005                                  
 zipcode              | 78207                                
only sho

#### Department

In [33]:
dept.show(1, vertical=True)

-RECORD 0----------------------------------
 dept_division          | 311 Call Center  
 dept_name              | Customer Service 
 standardized_dept_name | Customer Service 
 dept_subject_to_SLA    | YES              
only showing top 1 row



In [36]:
dept.dtypes

[('dept_division', 'string'),
 ('dept_name', 'string'),
 ('standardized_dept_name', 'string'),
 ('dept_subject_to_SLA', 'string')]

In [37]:
# convert 'dept_subject_to_SLA' to boolean
dept = dept.withColumn('dept_subject_to_SLA', expr('dept_subject_to_SLA == "YES"'))

In [38]:
# good to go?
dept.show(1, vertical=True)

-RECORD 0----------------------------------
 dept_division          | 311 Call Center  
 dept_name              | Customer Service 
 standardized_dept_name | Customer Service 
 dept_subject_to_SLA    | true             
only showing top 1 row



#### Source

In [39]:
source.show(1, vertical=True)

-RECORD 0---------------------------
 source_id       | 100137           
 source_username | Merlene Blodgett 
only showing top 1 row



In [40]:
source.dtypes

[('source_id', 'string'), ('source_username', 'string')]

In [ ]:
# good to go

### Data Wrangling

1. How old is the latest (in terms of days past SLA) currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?

In [54]:
# filter to only look at currently open cases
# use a datediff, or sort by days_open with ascending=False

In [55]:
#create three new columns 'case_age', 'days_to_closed', 'case_lifetime'

case = (
    case.withColumn(
        "case_age", datediff(current_timestamp(), "case_opened_date")
    )
    .withColumn(
        "days_to_closed", datediff("case_closed_date", "case_opened_date")
    )
    .withColumn(
        "case_lifetime",
        when(expr("! case_closed"), col("case_age")).otherwise(
            col("days_to_closed")
        ),
    )
)

In [65]:
# filtering where case is still open
# sorting by age of case, in descending order
# not truncating any values
# showing vertically, the top 1 case
case.filter(case.case_closed == 'false').sort(desc('case_age')).show(1, False, True)

-RECORD 0------------------------------------------------------
 case_id              | 1013225646                             
 case_opened_date     | 2017-01-01 13:48:00                    
 case_closed_date     | null                                   
 case_due_date        | 2017-01-17 08:30:00                    
 case_late            | true                                   
 num_days_late        | 348.6458333                            
 case_closed          | false                                  
 dept_division        | Code Enforcement                       
 service_request_type | No Address Posted                      
 case_days            | 15.77859954                            
 case_status          | Open                                   
 source_id            | svcCRMSS                               
 request_address      | 7299  shadow ridge, san antonio, 78250 
 council_district     | 006                                    
 zipcode              | 78250           

2. How many Stray Animal cases are there?

In [74]:
case.select(case.service_request_type).show(5)

+--------------------+
|service_request_type|
+--------------------+
|        Stray Animal|
|Removal Of Obstru...|
|Removal Of Obstru...|
|Front Or Side Yar...|
|Animal Cruelty(Cr...|
+--------------------+
only showing top 5 rows



In [78]:
case.filter(case.service_request_type == 'Stray Animal').count()

26760

3. How many service requests that are assigned to the Field Operations department (`dept_division`) are not classified as "Officer Standby" request type (`service_request_type`)?

4. Convert the `council_district` column to a string column.

5. Extract the year from the `case_closed_date` column.

6. Convert `num_days_late` from days to hours in new columns `num_hours_late`.

7. Join the case data with the source and department data.

8. Are there any cases that do not have a request source?

9. What are the top 10 service request types in terms of number of requests?

10. What are the top 10 service request types in terms of average days late?

11. Does number of days late depend on department?


12. How do number of days late depend on department and request type?